In [1]:
'''
Reader for EDF+ files.
TODO:
 - add support for log-transformed channels:
   http://www.edfplus.info/specs/edffloat.html and test with
   data generated with
   http://www.edfplus.info/downloads/software/NeuroLoopGain.zip.
 - check annotations with Schalk's Physiobank data.
Copyright (c) 2012 Boris Reuderink.
'''

import re, datetime, operator, logging
import numpy as np
from collections import namedtuple

EVENT_CHANNEL = 'EDF Annotations'
log = logging.getLogger(__name__)

class EDFEndOfData: pass


def tal(tal_str):
  '''Return a list with (onset, duration, annotation) tuples for an EDF+ TAL
  stream.
  '''
  exp = '(?P<onset>[+\-]\d+(?:\.\d*)?)' + \
    '(?:\x15(?P<duration>\d+(?:\.\d*)?))?' + \
    '(\x14(?P<annotation>[^\x00]*))?' + \
    '(?:\x14\x00)'

  def annotation_to_list(annotation):
    return unicode(annotation, 'utf-8').split('\x14') if annotation else []

  def parse(dic):
    return (
      float(dic['onset']),
      float(dic['duration']) if dic['duration'] else 0.,
      annotation_to_list(dic['annotation']))

  return [parse(m.groupdict()) for m in re.finditer(exp, tal_str)]


def edf_header(f):
  h = {}
  assert f.tell() == 0  # check file position
  assert f.read(8) == '0       '

  # recording info)
  h['local_subject_id'] = f.read(80).strip()
  h['local_recording_id'] = f.read(80).strip()

  # parse timestamp
  (day, month, year) = [int(x) for x in re.findall('(\d+)', f.read(8))]
  (hour, minute, sec)= [int(x) for x in re.findall('(\d+)', f.read(8))]
  h['date_time'] = str(datetime.datetime(year + 2000, month, day,
    hour, minute, sec))

  # misc
  header_nbytes = int(f.read(8))
  subtype = f.read(44)[:5]
  h['EDF+'] = subtype in ['EDF+C', 'EDF+D']
  h['contiguous'] = subtype != 'EDF+D'
  h['n_records'] = int(f.read(8))
  h['record_length'] = float(f.read(8))  # in seconds
  nchannels = h['n_channels'] = int(f.read(4))

  # read channel info
  channels = range(h['n_channels'])
  h['label'] = [f.read(16).strip() for n in channels]
  h['transducer_type'] = [f.read(80).strip() for n in channels]
  h['units'] = [f.read(8).strip() for n in channels]
  h['physical_min'] = np.asarray([float(f.read(8)) for n in channels])
  h['physical_max'] = np.asarray([float(f.read(8)) for n in channels])
  h['digital_min'] = np.asarray([float(f.read(8)) for n in channels])
  h['digital_max'] = np.asarray([float(f.read(8)) for n in channels])
  h['prefiltering'] = [f.read(80).strip() for n in channels]
  h['n_samples_per_record'] = [int(f.read(8)) for n in channels]
  f.read(32 * nchannels)  # reserved

  assert f.tell() == header_nbytes
  return h


class BaseEDFReader:
  def __init__(self, file):
    self.file = file


  def read_header(self):
    self.header = h = edf_header(self.file)

    # calculate ranges for rescaling
    self.dig_min = h['digital_min']
    self.phys_min = h['physical_min']
    phys_range = h['physical_max'] - h['physical_min']
    dig_range = h['digital_max'] - h['digital_min']
    assert np.all(phys_range > 0)
    assert np.all(dig_range > 0)
    self.gain = phys_range / dig_range


  def read_raw_record(self):
    '''Read a record with data and return a list containing arrays with raw
    bytes.
    '''
    result = []
    for nsamp in self.header['n_samples_per_record']:
      samples = self.file.read(nsamp * 2)
      if len(samples) != nsamp * 2:
        raise EDFEndOfData
      result.append(samples)
    return result


  def convert_record(self, raw_record):
    '''Convert a raw record to a (time, signals, events) tuple based on
    information in the header.
    '''
    h = self.header
    dig_min, phys_min, gain = self.dig_min, self.phys_min, self.gain
    time = float('nan')
    signals = []
    events = []
    for (i, samples) in enumerate(raw_record):
      if h['label'][i] == EVENT_CHANNEL:
        ann = tal(samples)
        time = ann[0][0]
        events.extend(ann[1:])
        # print(i, samples)
        # exit()
      else:
        # 2-byte little-endian integers
        dig = np.fromstring(samples, '<i2').astype(np.float32)
        phys = (dig - dig_min[i]) * gain[i] + phys_min[i]
        signals.append(phys)

    return time, signals, events


  def read_record(self):
    return self.convert_record(self.read_raw_record())


  def records(self):
    '''
    Record generator.
    '''
    try:
      while True:
        yield self.read_record()
    except EDFEndOfData:
      pass


def load_edf(edffile):
  '''Load an EDF+ file.
  Very basic reader for EDF and EDF+ files. While BaseEDFReader does support
  exotic features like non-homogeneous sample rates and loading only parts of
  the stream, load_edf expects a single fixed sample rate for all channels and
  tries to load the whole file.
  Parameters
  ----------
  edffile : file-like object or string
  Returns
  -------
  Named tuple with the fields:
    X : NumPy array with shape p by n.
      Raw recording of n samples in p dimensions.
    sample_rate : float
      The sample rate of the recording. Note that mixed sample-rates are not
      supported.
    sens_lab : list of length p with strings
      The labels of the sensors used to record X.
    time : NumPy array with length n
      The time offset in the recording for each sample.
    annotations : a list with tuples
      EDF+ annotations are stored in (start, duration, description) tuples.
      start : float
        Indicates the start of the event in seconds.
      duration : float
        Indicates the duration of the event in seconds.
      description : list with strings
        Contains (multiple?) descriptions of the annotation event.
  '''
  if isinstance(edffile, basestring):
    with open(edffile, 'rb') as f:
      return load_edf(f)  # convert filename to file

  reader = BaseEDFReader(edffile)
  reader.read_header()

  h = reader.header
  log.debug('EDF header: %s' % h)

  # get sample rate info
  nsamp = np.unique(
    [n for (l, n) in zip(h['label'], h['n_samples_per_record'])
    if l != EVENT_CHANNEL])
  assert nsamp.size == 1, 'Multiple sample rates not supported!'
  sample_rate = float(nsamp[0]) / h['record_length']

  rectime, X, annotations = zip(*reader.records())
  X = np.hstack(X)
  annotations = reduce(operator.add, annotations)
  chan_lab = [lab for lab in reader.header['label'] if lab != EVENT_CHANNEL]

  # create timestamps
  if reader.header['contiguous']:
    time = np.arange(X.shape[1]) / sample_rate
  else:
    reclen = reader.header['record_length']
    within_rec_time = np.linspace(0, reclen, nsamp, endpoint=False)
    time = np.hstack([t + within_rec_time for t in rectime])

  tup = namedtuple('EDF', 'X sample_rate chan_lab time annotations')
  return tup(X, sample_rate, chan_lab, time, annotations)

In [2]:
import numpy as np
channel_map=np.zeros(shape=(10,11),dtype=np.int32)
channel_map[0,4:7]=range(22,25)
channel_map[1,3:8]=range(25,30)
channel_map[2,1:10]=range(30,39)
channel_map[3,1]=39
channel_map[3,9]=40
channel_map[3,2:9]=range(1,8)
channel_map[4,0]=43
channel_map[4,1]=41
channel_map[4,9]=42
channel_map[4,10]=44
channel_map[4,2:9]=range(8,15)
channel_map[5,1]=45
channel_map[5,9]=46
channel_map[5,2:9]=range(15,22)
channel_map[6,1:10]=range(47,56)
channel_map[9,5]=64
channel_map[8,4:7]=range(61,64)
channel_map[7,3:8]=range(56,61)
channel_map

array([[ 0,  0,  0,  0, 22, 23, 24,  0,  0,  0,  0],
       [ 0,  0,  0, 25, 26, 27, 28, 29,  0,  0,  0],
       [ 0, 30, 31, 32, 33, 34, 35, 36, 37, 38,  0],
       [ 0, 39,  1,  2,  3,  4,  5,  6,  7, 40,  0],
       [43, 41,  8,  9, 10, 11, 12, 13, 14, 42, 44],
       [ 0, 45, 15, 16, 17, 18, 19, 20, 21, 46,  0],
       [ 0, 47, 48, 49, 50, 51, 52, 53, 54, 55,  0],
       [ 0,  0,  0, 56, 57, 58, 59, 60,  0,  0,  0],
       [ 0,  0,  0,  0, 61, 62, 63,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0, 64,  0,  0,  0,  0,  0]], dtype=int32)

In [3]:
win_size=10
sample_win_matrix_0=[] #[num,time_steps,10,11]
sample_win_matrix_1=[]
def label(name):
    if(name[0]=='T0'):
        return 0
    if((name[0]=='T1')|(name[0]=='T2')):
        return 1
    print name
    print "problem!"
    return 0
    
for person in range(1,90): 
    print person
    if person==89: #ruined data
        continue;
        
    if (person<=9):
        index_person="00"+str(person)
    elif (person<=99):
        index_person="0"+str(person)

    data=load_edf("../S"+index_person+"R01.edf")
    X=data[0]
    num_wins=int((X.shape[1]-win_size)/5)+1
    
    num_win=0
    while(num_win<num_wins):
        win=X[:,num_win*5:num_win*5+win_size]
        win_matrix=[] #np.zeros(shape=(win_size,10,11),dtype=np.float32)
        for i in range(win_size):
            matrix_i=np.zeros(shape=(10,11),dtype=np.float32)
            for x in range(10):
                for y in range(11):
                    if (channel_map[x,y]!=0):
                        matrix_i[x,y]=win[channel_map[x,y]-1,i]
                    else :
                        matrix_i[x,y]=0.0
            win_matrix.append(matrix_i)
        sample_win_matrix_0.append(win_matrix)
               
        num_win+=1
   

#     ### task 1:open and close left or right fist
#     data=load_edf("../S"+index_person+"R03.edf")
#     X=data[0]
#     time=data[3]
#     annotation=data[4]
#     i=0
#     i_max=X.shape[1]
#     for annot in annotation:
#         if (annot[0]==0.0):
#             while(time[i]!=0.0):
#                 i+=1
#         while(time[i]<annot[0]):
#             i+=1
#         while((time[i+win_size-1]<=annot[0]+annot[1])&(time[i+win_size-1]>annot[0])):
#             win=X[:,i:i+win_size]
#             win_matrix=[] 
#             for j in range(win_size):
#                 matrix_j=np.zeros(shape=(10,11),dtype=np.float32)
#                 for x in range(10):
#                     for y in range(11):
#                         if(channel_map[x,y]!=0):
#                              matrix_j[x,y]=win[channel_map[x,y]-1,j]
#                         else :
#                              matrix_j[x,y]=0.0
#                 win_matrix.append(matrix_j)
#             sample_win_matrix_0.append(win_matrix)
#             i+=5
#             if(i+win_size-1>=i_max):
#                 break
            


    #task 2 motion imagery 
    data=load_edf("../S"+index_person+"R04.edf")
    X=data[0]
    time=data[3]
    annotation=data[4]
    data=load_edf("../S"+index_person+"R08.edf")
    X=np.concatenate((X,data[0]),axis=1)
    time=np.concatenate((time,data[3]),axis=0)
    annotation+=data[4]
    data=load_edf("../S"+index_person+"R12.edf")
    X=np.concatenate((X,data[0]),axis=1)
    time=np.concatenate((time,data[3]),axis=0)
    annotation+=data[4]

    i=0
    i_max=X.shape[1]
    for annot in annotation:
        if (annot[0]==0.0):
            while(time[i]!=0.0):
                i+=1
        while(time[i]<annot[0]):
            i+=1
        while((time[i+win_size-1]<=annot[0]+annot[1])&(time[i+win_size-1]>annot[0])):
            win=X[:,i:i+win_size]
            win_matrix=[] 
            for j in range(win_size):
                matrix_j=np.zeros(shape=(10,11),dtype=np.float32)
                for x in range(10):
                    for y in range(11):
                        if(channel_map[x,y]!=0):
                            matrix_j[x,y]=win[channel_map[x,y]-1,j]
                        else:
                            matrix_j[x,y]=0.0
                win_matrix.append(matrix_j)
            if(label(annot[2])==0):
                sample_win_matrix_0.append(win_matrix)
            else :
                sample_win_matrix_1.append(win_matrix)
            
            i+=5
            if(i+win_size-1>=i_max):
                break

                
sample_win_matrix_0=np.array(sample_win_matrix_0)
sample_win_matrix_1=np.array(sample_win_matrix_1)

1


/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:130: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead


2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89


In [21]:
#cascade models
import tensorflow as tf
#variables
num_time_steps=10
batch_size=1000


tf.reset_default_graph()
labels=tf.placeholder(tf.int32)


filter1=tf.get_variable(name='filter1',shape=[3,3,1,32],dtype=tf.float32)
filter2=tf.get_variable(name='filter2',shape=[3,3,32,64],dtype=tf.float32)
filter3=tf.get_variable(name='filter3',shape=[3,3,64,128],dtype=tf.float32)
with tf.variable_scope("fn") as scope: 
        weights=tf.get_variable(name='weights',shape=[14080,1024],dtype=tf.float32)
        weights=tf.get_variable(name='biases',shape=[1024],dtype=tf.float32)
#weights_channel=tf.Variable(tf.random_uniform((1,1,1,1),minval=0.0000001,maxval=2.0,dtype=tf.float32))


channel_matrix=[]
for i in range(0,num_time_steps):
    channel_matrix.append(tf.placeholder(tf.float32,shape=(1000,10,11,1)))
cnn_output=[]

for cm in channel_matrix:
#     cm_weighted=tf.layers.conv2d(
#         input=cm,
#         filter=weights_channel, #[filter_height, filter_width, in_channels, out_channels]
#         strides=[1,1,1,1],           #[1, stride, stride, 1]
#         padding="SAME",
#         #use_cudnn_on_gpu=True,
#         data_format='channels_last'
#     )
    conv1=tf.nn.conv2d(
        input=cm, #=cm_weighted
        filter=filter1,
        strides=[1,1,1,1],           #[1, stride, stride, 1]
        padding="SAME",
        use_cudnn_on_gpu=True,
        data_format='NHWC',
    )
    conv1_drop=tf.nn.tanh(tf.nn.dropout(conv1,keep_prob=0.5))
    
    conv2=tf.nn.conv2d(
        input=conv1_drop, #=cm_weighted
        filter=filter2,
        strides=[1,1,1,1],           #[1, stride, stride, 1]
        padding="SAME",
        use_cudnn_on_gpu=True,
        data_format='NHWC',
    )
    
    conv2_drop=tf.nn.tanh(tf.nn.dropout(conv2,keep_prob=0.5))
    
    conv3=tf.nn.conv2d(
        input=conv2_drop, #=cm_weighted
        filter=filter3,
        strides=[1,1,1,1],           #[1, stride, stride, 1]
        padding="SAME",
        use_cudnn_on_gpu=True,
        data_format='NHWC',
    )

    #10 11 1 
    #f1: 8 9 32
    #f2: 6 7 64
    #f3: 4 5 128
    conv3_drop=tf.nn.tanh(tf.nn.dropout(conv3,keep_prob=0.5))
    
    conv3_flat=tf.contrib.layers.flatten(conv3_drop)
    
    with tf.variable_scope("fn") as scope: 
        
        fn=tf.contrib.layers.fully_connected(
            inputs=conv3_flat,
            num_outputs=1024,
            activation_fn=tf.nn.tanh,
            normalizer_fn=None,
            normalizer_params=None,
            weights_initializer=tf.contrib.layers.xavier_initializer(),
            weights_regularizer=None,
            biases_initializer=tf.zeros_initializer(),
            biases_regularizer=None,
            reuse=True,
            variables_collections=None,
            outputs_collections=None,
            trainable=True,
            scope=scope
        )
    
    
    fn_drop=tf.nn.dropout(fn,keep_prob=0.5)
    
    cnn_output.append(fn_drop)


#LSTM
cell = tf.nn.rnn_cell.BasicLSTMCell(
    num_units=64, 
    forget_bias=1.0,
    activation=tf.nn.tanh,
    state_is_tuple=True)

cell_drop=tf.contrib.rnn.DropoutWrapper(cell, output_keep_prob=0.5)

cell2 = tf.nn.rnn_cell.BasicLSTMCell(
    num_units=64, 
    forget_bias=1.0, 
    activation=tf.nn.tanh,
    state_is_tuple=True)

cell2_drop=tf.contrib.rnn.DropoutWrapper(cell2, output_keep_prob=0.5)

lstm_cell = tf.contrib.rnn.MultiRNNCell(cells=[cell_drop,cell2_drop], state_is_tuple=True)


output,final_state= tf.nn.static_rnn(
    cell=lstm_cell,
    #cell=cell2,
    inputs=cnn_output,
    #A length T list of inputs,
    #each a Tensor of shape [batch_size, input_size], 
    #or a nested tuple of such elements.
    initial_state=None,
    dtype=tf.float32,
    sequence_length=tf.convert_to_tensor(np.repeat(num_time_steps,batch_size)),
    scope=None
)

fn2=tf.contrib.layers.fully_connected(
    inputs=output[-1],#output is just the state vectors for basiccells in tf
    num_outputs=16,
    activation_fn=tf.nn.tanh,
    normalizer_fn=None,
    normalizer_params=None,
    weights_initializer=tf.contrib.layers.xavier_initializer(),
    weights_regularizer=None,
    biases_initializer=tf.zeros_initializer(),
    biases_regularizer=None,
    reuse=None,
    variables_collections=None,
    outputs_collections=None,
    trainable=True,
    scope=None)

fn2_drop=tf.nn.dropout(fn2,keep_prob=0.5)

W=tf.get_variable(name='W',shape=[16,2],dtype=tf.float32)
logits=tf.matmul(fn2_drop,W)
crossent = tf.nn.sparse_softmax_cross_entropy_with_logits(
    labels=labels, logits=logits)
train_loss = (tf.reduce_sum(crossent)/batch_size)

# Optimization
params = tf.trainable_variables()# the list of params to be trained
print params #check the trainable params, including encoder's ?

optimizer = tf.train.AdamOptimizer(learning_rate=0.01,epsilon=0.001)
gradients=optimizer.compute_gradients(
    train_loss,
    var_list=None,
    aggregation_method=None,
    colocate_gradients_with_ops=False,
    grad_loss=None
)

update= optimizer.apply_gradients(gradients)


tf.summary.scalar('LLLLoss', train_loss)
for i,g in enumerate(gradients):
     tf.summary.histogram('GGGGradients'+str(i), g[0])
merged = tf.summary.merge_all()
sess=tf.Session()
train_writer = tf.summary.FileWriter('./summary',sess.graph)

init = tf.global_variables_initializer()
sess.run(init)

[<tf.Variable 'filter1:0' shape=(3, 3, 1, 32) dtype=float32_ref>, <tf.Variable 'filter2:0' shape=(3, 3, 32, 64) dtype=float32_ref>, <tf.Variable 'filter3:0' shape=(3, 3, 64, 128) dtype=float32_ref>, <tf.Variable 'fn/weights:0' shape=(14080, 1024) dtype=float32_ref>, <tf.Variable 'fn/biases:0' shape=(1024,) dtype=float32_ref>, <tf.Variable 'rnn/multi_rnn_cell/cell_0/basic_lstm_cell/kernel:0' shape=(1088, 256) dtype=float32_ref>, <tf.Variable 'rnn/multi_rnn_cell/cell_0/basic_lstm_cell/bias:0' shape=(256,) dtype=float32_ref>, <tf.Variable 'rnn/multi_rnn_cell/cell_1/basic_lstm_cell/kernel:0' shape=(128, 256) dtype=float32_ref>, <tf.Variable 'rnn/multi_rnn_cell/cell_1/basic_lstm_cell/bias:0' shape=(256,) dtype=float32_ref>, <tf.Variable 'fully_connected/weights:0' shape=(64, 16) dtype=float32_ref>, <tf.Variable 'fully_connected/biases:0' shape=(16,) dtype=float32_ref>, <tf.Variable 'W:0' shape=(16, 2) dtype=float32_ref>]


In [17]:
print sample_win_matrix_0.shape
print sample_win_matrix_0[0,0,:,:]

(1027914, 10, 10, 11)
[[  0.   0.   0.   0. -49. -44. -29.   0.   0.   0.   0.]
 [  0.   0.   0. -73. -67. -41. -39. -62.   0.   0.   0.]
 [  0. -39. -42. -16. -36. -38. -46. -60. -69. -25.   0.]
 [  0. -47. -16. -29.   2.  22. -12. -23. -46. -80.   0.]
 [-52. -71. -36. -26. -18.  -4.  -4. -20. -22. -32.   1.]
 [  0. -58. -33. -39. -23. -12. -16. -27. -25. -30.   0.]
 [  0. -56. -43. -12. -18. -31. -26. -37. -33. -30.   0.]
 [  0.   0.   0. -56. -52. -35. -22. -33.   0.   0.   0.]
 [  0.   0.   0.   0. -53. -21. -11.   0.   0.   0.   0.]
 [  0.   0.   0.   0.   0.  15.   0.   0.   0.   0.   0.]]


In [23]:
sess.run(init)

num_0=sample_win_matrix_0.shape[0]
num_1=sample_win_matrix_1.shape[0]
num_window=num_0+num_1
num_batch=int(num_window/batch_size)
i_summary=0

for i in range(num_batch):
    
    #sampling training 
    random_index=range(num_0)
    np.random.shuffle(random_index)
    batch_0=sample_win_matrix_0[random_index[:int(batch_size/2)]]
    
    random_index=range(num_1)
    np.random.shuffle(random_index)
    batch_1=sample_win_matrix_1[random_index[:int(batch_size/2)]]
    
    batch=np.concatenate((batch_0,batch_1),axis=0)
    random_index=range(batch_size)
    np.random.shuffle(random_index)
    batch=batch[random_index]
    batch=np.expand_dims(batch,axis=4)
    
    batch_labels=[]
    for i in random_index:
        if i < int(batch_size/2):
            batch_labels.append(0)
        else :
            batch_labels.append(1)

    
    #run
    _,summary=sess.run([update,merged],{labels:batch_labels,
                      channel_matrix[0]:batch[:,0,:,:,:],
                      channel_matrix[1]:batch[:,1,:,:,:], 
                      channel_matrix[2]:batch[:,2,:,:,:],
                      channel_matrix[3]:batch[:,3,:,:,:], 
                      channel_matrix[4]:batch[:,4,:,:,:], 
                      channel_matrix[5]:batch[:,5,:,:,:], 
                      channel_matrix[6]:batch[:,6,:,:,:], 
                      channel_matrix[7]:batch[:,7,:,:,:], 
                      channel_matrix[8]:batch[:,8,:,:,:], 
                      channel_matrix[9]:batch[:,9,:,:,:]})
    train_writer.add_summary(summary, i_summary)
    i_summary+=1

In [24]:
print sess.run(weights)

[-0.03516394  0.04280772 -0.09934779 ...  0.07924265 -0.07669201
 -0.06150619]


In [25]:
#test
result=[]
labels_test=[]
for i in range(5):
    
    #sampling training 
    random_index=range(num_0)
    np.random.shuffle(random_index)
    batch_0=sample_win_matrix_0[random_index[:int(batch_size/2)]]
    
    random_index=range(num_1)
    np.random.shuffle(random_index)
    batch_1=sample_win_matrix_1[random_index[:int(batch_size/2)]]
    
    batch=np.concatenate((batch_0,batch_1),axis=0)
    random_index=range(batch_size)
    np.random.shuffle(random_index)
    batch=batch[random_index]
    batch=np.expand_dims(batch,axis=4)
    
    batch_labels=[]
    for i in random_index:
        if i < int(batch_size/2):
            batch_labels.append(0)
        else :
            batch_labels.append(1)
    labels_test+=batch_labels
    #run 
    r=sess.run([logits],{channel_matrix[0]:batch[:,0,:,:,:],
                      channel_matrix[1]:batch[:,1,:,:,:], 
                      channel_matrix[2]:batch[:,2,:,:,:],
                      channel_matrix[3]:batch[:,3,:,:,:], 
                      channel_matrix[4]:batch[:,4,:,:,:], 
                      channel_matrix[5]:batch[:,5,:,:,:], 
                      channel_matrix[6]:batch[:,6,:,:,:], 
                      channel_matrix[7]:batch[:,7,:,:,:], 
                      channel_matrix[8]:batch[:,8,:,:,:], 
                      channel_matrix[9]:batch[:,9,:,:,:]})
    result+=r
print result[0]
    
result=np.concatenate(result,axis=0)
result=np.array(result)
result=np.argmax(result,axis=1)
labels_test=np.array(labels_test)
from sklearn.metrics import classification_report
print classification_report(labels_test,result)

[[ 0.08952504  0.21811095]
 [ 0.10135216  0.3889083 ]
 [-0.17367351 -0.379184  ]
 ...
 [ 0.11639372 -0.37282267]
 [-0.03696988  0.02415639]
 [-0.01761211 -0.01787742]]
             precision    recall  f1-score   support

          0       0.60      0.55      0.57      2500
          1       0.58      0.64      0.61      2500

avg / total       0.59      0.59      0.59      5000



In [ ]:
X_total=[]
y_total=[]
channel_available=[60,62,31,35,48,52,8,12]
for person in range(1,93): 
    if person==89: #ruined data
        continue;
        
    if (person<=9):
        index_person="00"+str(person)
    elif (person<=99):
        index_person="0"+str(person)
        
    #baseline
    data=load_edf("../S"+index_person+"R01.edf")
    X=data[0][channel_available]
    num_wins=int((X.shape[1]-win_size)/5)+1
    
    num_win=0
    while(num_win<num_wins):
        X_total.append(np.ndarray.flatten(X[:,num_win*5:num_win*5+win_size]))     
        y_total.append(0)
               
        num_win+=1
   

    ### task 1:open and close left or right fist
    data=load_edf("../S"+index_person+"R03.edf")
    X=data[0][channel_available]
    time=data[3]
    annotation=data[4]
    i=0
    i_max=X.shape[1]
    for annot in annotation:
        if (annot[0]==0.0):
            while(time[i]!=0.0):
                i+=1
        while(time[i]<annot[0]):
            i+=1
        while((time[i+win_size-1]<=annot[0]+annot[1])&(time[i+win_size-1]>annot[0])):
            X_total.append(np.ndarray.flatten(X[:,i:i+win_size]))
            y_total.append(0)
            i+=5
            if(i+win_size-1>=i_max):
                break
            


    #task 2 motion imagery 
    data=load_edf("../S"+index_person+"R04.edf")
    X=data[0][channel_available]
    time=data[3]
    annotation=data[4]
    data=load_edf("../S"+index_person+"R08.edf")
    X=np.concatenate((X,data[0][channel_available]),axis=1)
    time=np.concatenate((time,data[3]),axis=0)
    annotation+=data[4]
    data=load_edf("../S"+index_person+"R12.edf")
    X=np.concatenate((X,data[0][channel_available]),axis=1)
    time=np.concatenate((time,data[3]),axis=0)
    annotation+=data[4]

    i=0
    i_max=X.shape[1]
    for annot in annotation:
        if (annot[0]==0.0):
            while(time[i]!=0.0):
                i+=1
        while(time[i]<annot[0]):
            i+=1
        while((time[i+win_size-1]<=annot[0]+annot[1])&(time[i+win_size-1]>annot[0])):
            X_total.append(np.ndarray.flatten(X[:,i:i+win_size]))       
            y_total.append(label(annot[2]))
            
            i+=5
            if(i+win_size-1>=i_max):
                break

#shuffle the data 
X_total=np.array(X_total)
y_total=np.array(y_total)
num_window=X_total.shape[0]
random_index=range(num_window)
np.random.shuffle(random_index)
X_total=X_total[random_index]
y_total=y_total[random_index]
X_train=X_total[:int(num_window/3*2),:]
y_train=y_total[:int(num_window/3*2)]
X_test=X_total[int(num_window/3*2):,:]
y_test=y_total[int(num_window/3*2):]


#LDA+ML
#is there online training in sklearn ? =.=
from sklearn.decomposition import IncrementalPCA as PCA
from sklearn.linear_model import SGDClassifier as SGD
#dim initial= 10*8=80
pca=PCA(n_components=10,batch_size=100)
pca.fit(X_train)
x_train=pca.transform(X_train)

sgd=SGD()
num_window=X_train.shape[0]
num_batch=int(num_window/100)-1
for i in range(num_batch):
    X_batch=x_train[i*100:(i+1)*100,:]
    y_batch=y_train[i*100:(i+1)*100]
    sgd=sgd.partial_fit(x_batch,y_batch)

x_test=pca.transform(X_test)
result=sgd.predict(x_test)
from sklearn.metrics import classification_report
print classification_report(np.array(y_test),np.array(result))


In [2]:
import tensorflow as tf
#variables
tf.reset_default_graph()
batch_size=222
num_channels=8
num_frequency=22

channel_frequency_input=tf.placeholder(tf.float32,shape=(222,num_channels,num_frequency,1)) #[batch, in_height=channels, in_width=frequency, in_channels=1]
encoder_labels=tf.placeholder(tf.int32)
# filter1=tf.Variable(tf.random_uniform((2,3,1,32),minval=0.0000001,maxval=0.00001,dtype=tf.float32))
# filter2=tf.Variable(tf.random_uniform((2,3,32,16),minval=0.0000001,maxval=0.00001,dtype=tf.float32))
# filter3=tf.Variable(tf.random_uniform((4,6,16,8),minval=0.0000001,maxval=0.00001,dtype=tf.float32))
filter1=tf.Variable(tf.random_uniform((1,2,1,32),minval=0.0000001,maxval=0.00001,dtype=tf.float32))
filter2=tf.Variable(tf.random_uniform((1,2,32,16),minval=0.0000001,maxval=0.00001,dtype=tf.float32))
filter3=tf.Variable(tf.random_uniform((8,8,16,8),minval=0.0000001,maxval=0.00001,dtype=tf.float32))
# filter1=tf.Variable(tf.random_uniform((3,3,1,16),minval=0.0000001,maxval=0.00001,dtype=tf.float32))
# filter2=tf.Variable(tf.random_uniform((4,8,16,8),minval=0.0000001,maxval=0.00001,dtype=tf.float32))

conv1=tf.nn.conv2d(
    input=channel_frequency_input,
    filter=filter1, #[filter_height, filter_width, in_channels, out_channels]
    strides=[1,1,2,1],           #[1, stride, stride, 1]
    padding="VALID",
    #use_cudnn_on_gpu=True,
    data_format='NHWC'
)

pool1=tf.nn.max_pool(
    value=conv1,
    ksize=[1,1,2,1],
    strides=[1,1,1,1],
    padding='VALID',
    data_format='NHWC'
)

conv2=tf.nn.conv2d(
    input=pool1,
    filter=filter2, #[filter_height, filter_width, in_channels, out_channels]
    strides=[1,1,1,1],           #[1, stride, stride, 1]
    padding="VALID",
    #use_cudnn_on_gpu=True,
    data_format='NHWC'
)

pool2=tf.nn.max_pool(
    value=conv2,
    ksize=[1,1,2,1],
    strides=[1,1,1,1],
    padding='VALID',
    data_format='NHWC'
)

conv3=tf.nn.conv2d(
    input=pool2,
    filter=filter3, #[filter_height, filter_width, in_channels, out_channels]
    strides=[1,1,1,1],           #[1, stride, stride, 1]
    padding="VALID",
    #use_cudnn_on_gpu=True,
    data_format='NHWC'
)

flat_cnn=tf.contrib.layers.flatten(conv3)
#flat_cnn=tf.contrib.layers.flatten(conv2)


output_cnn=tf.sigmoid(flat_cnn)
output_shape1=tf.size(output_cnn)

#LSTM
num_time_steps=20
channel_amplitude_input=[]
for i in range(0,num_time_steps):
    channel_amplitude_input.append(tf.placeholder(tf.float32,shape=(222,num_channels)))


cell = tf.nn.rnn_cell.BasicLSTMCell(
    num_units=16, 
    forget_bias=1.0, 
    state_is_tuple=True)

cell2 = tf.nn.rnn_cell.BasicLSTMCell(
    num_units=8, 
    forget_bias=1.0, 
    state_is_tuple=True)

lstm_cell = tf.contrib.rnn.MultiRNNCell(cells=[cell,cell2], state_is_tuple=True)


output,final_state= tf.nn.static_rnn(
    cell=lstm_cell,
    #cell=cell2,
    inputs=channel_amplitude_input,
    #A length T list of inputs,
    #each a Tensor of shape [batch_size, input_size], 
    #or a nested tuple of such elements.
    initial_state=None,
    dtype=tf.float32,
    sequence_length=tf.convert_to_tensor(np.repeat(num_time_steps,batch_size)),
    scope=None
)

output_shape2=tf.size(output[-1])
flat_lstm=tf.contrib.layers.flatten(output[-1])

#vec_concat=tf.concat([output_cnn,flat_lstm],axis=1)


W=tf.Variable(tf.random_uniform((8,2),minval=0.0000001,maxval=0.00001,dtype=tf.float32)) 
logits=tf.matmul(flat_lstm,W)
crossent = tf.nn.sparse_softmax_cross_entropy_with_logits(
    labels=encoder_labels, logits=logits)
train_loss = (tf.reduce_sum(crossent)/batch_size)

# Optimization
params = tf.trainable_variables()# the list of params to be trained
print params #check the trainable params, including encoder's ?

optimizer = tf.train.AdamOptimizer(learning_rate=0.001,epsilon=0.001)
gradients=optimizer.compute_gradients(
    train_loss,
    var_list=None,
    aggregation_method=None,
    colocate_gradients_with_ops=False,
    grad_loss=None
)

update= optimizer.apply_gradients(gradients)


tf.summary.scalar('LLLLoss', train_loss)
# for i,g in enumerate(gradients):
#     tf.summary.histogram('GGGGradients'+str(i), g[0])
merged = tf.summary.merge_all()
sess=tf.Session()
train_writer = tf.summary.FileWriter('./summary',sess.graph)

init = tf.global_variables_initializer()

# channel_amp=[]
# for i in range(10):
#     channel_amp.append(np.zeros(shape=(222,8),dtype="float32"))

# g,s,s2,l,_,summ=sess.run([gradients,output_shape1,output_shape2,train_loss,update,merged],{channel_frequency_input:np.zeros(shape=(222,8,12,1),dtype="float32"),\
#                           encoder_labels:np.repeat(1,222),
#                           channel_amplitude_input[0]:channel_amp[0],  
#                           channel_amplitude_input[1]:channel_amp[1], 
#                           channel_amplitude_input[2]:channel_amp[2],
#                           channel_amplitude_input[3]:channel_amp[3], 
#                           channel_amplitude_input[4]:channel_amp[4], 
#                           channel_amplitude_input[5]:channel_amp[5], 
#                           channel_amplitude_input[6]:channel_amp[6], 
#                           channel_amplitude_input[7]:channel_amp[7], 
#                           channel_amplitude_input[8]:channel_amp[8], 
#                           channel_amplitude_input[9]:channel_amp[9]})
# writer.add_summary(summ, 1)
# writer.add_summary(summ, 2)
# print type(g)
# print g[0]
# print s
# print s2
# print l

[<tf.Variable 'Variable:0' shape=(1, 2, 1, 32) dtype=float32_ref>, <tf.Variable 'Variable_1:0' shape=(1, 2, 32, 16) dtype=float32_ref>, <tf.Variable 'Variable_2:0' shape=(8, 8, 16, 8) dtype=float32_ref>, <tf.Variable 'rnn/multi_rnn_cell/cell_0/basic_lstm_cell/kernel:0' shape=(24, 64) dtype=float32_ref>, <tf.Variable 'rnn/multi_rnn_cell/cell_0/basic_lstm_cell/bias:0' shape=(64,) dtype=float32_ref>, <tf.Variable 'rnn/multi_rnn_cell/cell_1/basic_lstm_cell/kernel:0' shape=(24, 32) dtype=float32_ref>, <tf.Variable 'rnn/multi_rnn_cell/cell_1/basic_lstm_cell/bias:0' shape=(32,) dtype=float32_ref>, <tf.Variable 'Variable_3:0' shape=(8, 2) dtype=float32_ref>]
